In [1]:
import requests
import pandas as pd
import numpy as np

# Base URL for the openFDA Drug Event API
base_url = "https://api.fda.gov/drug/event.json"
params = {
    "limit": 100,  # Maximum number of records per request
    "skip": 0      # Starting point for pagination
}

# List to store extracted records
records = []
required_observations = 800  # Minimum requirement for your project
collected_records = 0

# Define function to extract specific fields from each record
def extract_fields(record):
    # Extract general report information
    report = {
        "safetyreportid": record.get("safetyreportid", ""),
        "serious": record.get("serious", ""),
        "seriousnessdeath": record.get("seriousnessdeath", ""),
        "receivedate": record.get("receivedate", ""),
        "reportercountry": record.get("primarysource", {}).get("reportercountry", ""),
        "reporterqualification": record.get("primarysource", {}).get("qualification", "")
    }
    
    # Extract patient demographics
    patient = record.get("patient", {})
    report["patient_age"] = patient.get("patientonsetage", None)
    report["patient_age_unit"] = patient.get("patientonsetageunit", "")
    report["patient_sex"] = patient.get("patientsex", "")
    
    # Extract reaction information
    reactions = patient.get("reaction", [])
    report["reaction_meddra"] = reactions[0].get("reactionmeddrapt", "") if reactions else ""
    
    # Extract drug information
    drugs = patient.get("drug", [])
    report["drug_name"] = drugs[0].get("medicinalproduct", "") if drugs else ""
    report["drug_characterization"] = drugs[0].get("drugcharacterization", "") if drugs else ""
    report["drug_admin_route"] = drugs[0].get("drugadministrationroute", "") if drugs else ""
    report["drug_indication"] = drugs[0].get("drugindication", "") if drugs else ""
    
    return report

# Loop to gather enough records
while collected_records < required_observations:
    # Make the API request
    response = requests.get(base_url, params=params)
    data = response.json()
    
    # Check if the response contains data
    if "results" in data:
        for record in data["results"]:
            # Extract specific fields and add to records list
            extracted_record = extract_fields(record)
            records.append(extracted_record)
            collected_records += 1
            
            # Stop if we've collected enough records
            if collected_records >= required_observations:
                break
        
        # Update pagination for next batch
        params["skip"] += params["limit"]
    else:
        print("No more data available.")
        break

# Convert records to a DataFrame
df = pd.DataFrame(records)



# Optional: Clean up and standardize categorical values
#df["patient_sex"] = df["patient_sex"].replace({1: "Male", 2: "Female"})
#df["serious"] = df["serious"].replace({1: "Serious", 0: "Non-Serious"})
#df["seriousnessdeath"] = df["seriousnessdeath"].replace({1: "Fatal", 0: "Non-Fatal"})


In [2]:
# The 'seriousnessdeath' column seems to be either 1 or NULL, so we should convert the column into a binary indicator!!!

# Convert the `seriousnessdeath` column to a binary indicator
# 1 indicates fatal (existing 1 values), and 0 indicates non-fatal (fill empty values with 0)

df['seriousnessdeath'] = df['seriousnessdeath'].apply(lambda x: 1 if x == 1 else 0)

# Display the updated column values to confirm changes
df['seriousnessdeath'].value_counts()


seriousnessdeath
0    800
Name: count, dtype: int64

In [3]:


# Step 2: Convert non-numeric values in 'patient_age_unit' to NaN, but leave numeric values untouched
df['patient_age_unit'] = pd.to_numeric(df['patient_age_unit'], errors='coerce')

patient_age_unit_mapping = {
    800: "Days",
    801: "Years",
    802: "Months",
    803: "Weeks",
    804: "Hours",
    np.nan: "NULL"
}

# Step 3: Map numeric codes to their corresponding labels
df['patient_age_unit'] = df['patient_age_unit'].map(patient_age_unit_mapping)


In [4]:
# Replace blank entries in 'patient_age' with NaN
#new_frame['patient_age'].replace("", np.nan, inplace=True)
df['patient_age'].replace("", np.nan, inplace=True)


# Drop rows where 'patient_age' is NaN
#new_frame.dropna(subset=['patient_age'], inplace=True)
df.dropna(subset=['patient_age'], inplace=True)


C:\Users\andym\AppData\Local\Temp\ipykernel_20108\3033652657.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['patient_age'].replace("", np.nan, inplace=True)


In [5]:
### Some of the missing datavalues are how the drugs are administered. To fill in those gaps, we could search for missing administation methods for drugs that 
#### exclusively have only 1 method of intake. That way, if X drug is only taken orally, then we can fill in "orally" whenever X drug is taken.
#### Note: The drug needs to have a large set of pervious variables that it was taken only 1 way. If there is even 1 other mentioned way to take it, then those
#### drugs are not filled. 

# Group by `drug_name` and check the unique count of `drug_admin_route` for each drug
admin_route_per_drug = df.groupby('drug_name')['drug_admin_route'].nunique()

# Filter drugs that are associated with only one administration route
single_route_drugs = admin_route_per_drug[admin_route_per_drug == 1].index.tolist()

# Check the overall distribution of drugs with single vs multiple routes
admin_route_per_drug.value_counts(), single_route_drugs[:10]  # Display sample of single-route drugs for review


(drug_admin_route
 1    139
 2     14
 3      3
 Name: count, dtype: int64,
 ['5-FLUOROURACIL',
  'ABATACEPT SUBQ INJECTION 125MG/ML',
  'ABRAXANE',
  'ACETAMINOPHEN.',
  'ADCIRCA',
  'ADIRO 100',
  'AFINITOR',
  'ALLOPURINOL.',
  'AMBRISENTAN',
  'AMIODARONE HCL'])

In [6]:
# Create a dictionary mapping drugs with a single administration route to their route value
single_route_mapping = df[df['drug_name'].isin(single_route_drugs)].dropna(subset=['drug_admin_route']) \
                       .groupby('drug_name')['drug_admin_route'].first().to_dict()

# Fill missing `drug_admin_route` based on the mapping, leaving multiple-route drugs as "Unknown" where missing
df['drug_admin_route'] = df.apply(
    lambda row: single_route_mapping.get(row['drug_name'], "Unknown") if pd.isna(row['drug_admin_route']) else row['drug_admin_route'],
    axis=1
)

# Verify if the missing values have been filled appropriately
df['drug_admin_route'].value_counts(dropna=False)


drug_admin_route
048    291
       123
042     31
058     23
065     17
015      9
055      7
030      6
033      6
059      4
041      2
050      1
040      1
062      1
067      1
Name: count, dtype: int64

In [7]:
df.to_csv("openFDA_AdverseDrugsSample.csv", index=False)
del df